# De-Identifiction of Students' Data
本程式碼用於對可能包含部分學生個人資訊的內容進行去識別/去連結化

## Step 0: 環境與資料設定
在此步驟中，將會先建構一份測試資料，並以變數形式指定欲去識別/去連結化的欄位與丟棄的欄位

### Step 0.0: 建構測試資料
- 本程式旨在建構一個彈性的去識別/去連結化方式，將不針對單資料進行分析，因此建構測試資料，方便後續改動
- 在測試資料中，包含了一個「學生識別碼」欄位 (`student_id`)，兩個「一般資料」欄位 (`data_1`, `data_2`)，以及一個「應丟棄資料」(`email`)

In [1]:
import pandas as pd
from IPython.display import display
test_data = pd.DataFrame({'student_id': [i for i in range(10)], 
                          'data_1': ['easy', 'hard', 'hard', 'easy', 'easy', 'easy', 'hard', 'hard', 'easy', 'easy'],
                          'data_2': ['high', 'low', 'high', 'high', 'low', 'high', 'low', 'high', 'high', 'low'],
                          'email': [f'{i}{i}@example.com' for i in range(10)]}).sample(frac=1)
display(test_data)

,student_id,data_1,data_2,email
1,1,hard,low,11@example.com
8,8,easy,high,88@example.com
2,2,hard,high,22@example.com
4,4,easy,low,44@example.com
5,5,easy,high,55@example.com
3,3,easy,high,33@example.com
7,7,hard,high,77@example.com
6,6,hard,low,66@example.com
9,9,easy,low,99@example.com
0,0,easy,high,00@example.com


### Step 0.1: 環境設定

In [2]:
import pandas as pd
from IPython.display import display
import random
import pickle

### Step 0.1: 資料設定
在此部分將會設定以下變數：
- `DEID_SOURCE`：欲重新編碼，進行去識別/去連結化的欄位
- `DEID_TARGET`：進行去識別/去連結化後，欲儲存的欄位
- `DROP_COLS`：欲丟棄的欄位
- `DEID_DICT_SIZE`：欲建立的編碼字典大小
- `DATA`：欲進行去識別/去連結化的資料，爲 `pandas.DataFrame` 格式
- `DATA_OUTPUT_PATH`: 欲儲存去識別/去連結化後的資料路徑

*本部分將以測試資料做說明

In [3]:
DEID_SOURCE = 'student_id'
DEID_TARGET = 'student_deid'
DROP_COLS = ['email']
DEID_DICT_SIZE = 10**5

DATA = test_data
DATA_OUTPUT_PATH = 'deid_data.pkl'

## Step 1: 建構對應字典
在本步驟中，將會建構一個一次性的隨機字典，用以編碼應去識別/去連結化的欄位

In [4]:
mapping_dict = {k: v for v, k in enumerate(random.sample(range(DEID_DICT_SIZE), DEID_DICT_SIZE))}

## Step 2: 進行去識別/去連結化
在本步驟中，將會進行去識別/去連結化的動作，並將結果輸出至指定位置

*由於並沒有指定隨機種子值，每次執行結果將會不同，儘可能保證隨機性

### Step 2.1 進行欄位替換

In [5]:
# ensure no missing values in the source column
assert not DATA[DEID_SOURCE].isna().any()

deid_results = DATA.assign(**{DEID_TARGET: DATA[DEID_SOURCE].astype(int).map(mapping_dict)}).drop(DEID_SOURCE, axis=1)

# ensure no missing values in the target column
assert not deid_results[DEID_TARGET].isna().any()

display(deid_results)

,data_1,data_2,email,student_deid
1,hard,low,11@example.com,910
8,easy,high,88@example.com,67592
2,hard,high,22@example.com,41693
4,easy,low,44@example.com,43523
5,easy,high,55@example.com,47838
3,easy,high,33@example.com,96838
7,hard,high,77@example.com,65348
6,hard,low,66@example.com,7224
9,easy,low,99@example.com,67748
0,easy,high,00@example.com,30511


### Step 2.2 丟棄額外欄位

In [6]:
deid_results = deid_results.drop(DROP_COLS, axis=1).reset_index(drop=True)
deid_results

,data_1,data_2,student_deid
0,hard,low,910
1,easy,high,67592
2,hard,high,41693
3,easy,low,43523
4,easy,high,47838
5,easy,high,96838
6,hard,high,65348
7,hard,low,7224
8,easy,low,67748
9,easy,high,30511


### Step 2.3 進行資料輸出

In [7]:
pickle.dump(deid_results, open(DATA_OUTPUT_PATH, "wb"))